# Match Events Populator

This module uses the 'Match Events Extractor' module in order to populate the table 'MatchEvent' in our database with
the events taken and reelaborated from the 'Match' table.

In [9]:
import nbimporter
import match_events_extractor
import db_utils

In [10]:
"""
Since not every match has the following attributes populated, for these matches we will use default values.
These values represent an average number of occurrences of each event for a given team in a soccer game.
"""

DEFAULT_SHOTON = DEFAULT_SHOTOFF = 6
DEFAULT_YELLOWS = 2
DEFAULT_FOULS = 13
DEFAULT_CORNERS = 5
DEFAULT_CROSSES = 17
DEFAULT_POSSESSION = 50

In [13]:
db_file = 'database.sqlite'

match_events = []

matches = db_utils.query(db_file, 'select * from Match', ())
for match in matches:
    match_id = match[0]
    home = match[7]
    away = match[8]
    homeG = match[9]
    awayG = match[10]
    shoton_xml = match[12]
    shotoff_xml = match[13]
    foulcommit_xml = match[14]
    card_xml = match[15]
    cross_xml = match[16]
    corner_xml = match[17]
    possession_xml = match[18]
    
    home_shoton, away_shoton = match_events_extractor.extract_shoton_from_xml(shoton_xml, str(home), str(away))
    if( home_shoton + away_shoton == 0 ):
        home_shoton = away_shoton = DEFAULT_SHOTON

    home_shotoff, away_shotoff = match_events_extractor.extract_shotoff_from_xml(shotoff_xml, str(home), str(away))
    if( home_shotoff + away_shotoff == 0 ):
        home_shotoff = away_shotoff = DEFAULT_SHOTOFF

    home_yellows, away_yellows = match_events_extractor.extract_yellows_from_xml(card_xml, str(home), str(away))
    if( home_yellows + away_yellows == 0 ):
        home_yellows = away_yellows = DEFAULT_YELLOWS

    home_reds, away_reds = match_events_extractor.extract_reds_from_xml(card_xml, str(home), str(away))

    home_fouls, away_fouls = match_events_extractor.extract_fouls_from_xml(foulcommit_xml, str(home), str(away))
    if( home_fouls + away_fouls == 0 ):
        home_fouls = away_fouls = DEFAULT_FOULS

    home_corners, away_corners = match_events_extractor.extract_corners_from_xml(corner_xml, str(home), str(away))
    if( home_corners + away_corners == 0 ):
        home_corners = away_corners = DEFAULT_CORNERS

    home_crosses, away_crosses = match_events_extractor.extract_crosses_from_xml(cross_xml, str(home), str(away))
    if( home_crosses + away_crosses == 0 ):
        home_crosses = away_crosses = DEFAULT_CROSSES

    home_possession, away_possession = match_events_extractor.extract_possession_from_xml(possession_xml)
    if( home_possession + away_possession == 0 ):
        home_possession = away_possession = DEFAULT_POSSESSION

    
    match_data = (match_id,homeG,awayG,home_shoton,away_shoton,home_shotoff, away_shotoff, 
                home_yellows,away_yellows,home_reds,away_reds,home_fouls,away_fouls,home_corners,away_corners,
                home_crosses, away_crosses,home_possession,away_possession)
    
    match_events.append(match_data)

In [14]:
#Write the match events in the MatchEvent table
sql = 'INSERT INTO MatchEvent VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)'
for item in match_events:
    db_utils.write(db_file, sql, item)